#   Crawling Berita CNN Indonesia

In [13]:
import builtwith

# Analisis teknologi yang digunakan
res = builtwith.parse('https://www.cnnindonesia.com/')
print(res)

{'cdn': ['CloudFlare'], 'databases': ['Firebase'], 'advertising-networks': ['Google AdSense'], 'tag-managers': ['Google Tag Manager'], 'javascript-frameworks': ['jQuery']}


In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

##  Fungsi untuk ambil kategori berita dan fungsi untuk mengambil data berita sebanyak 20 per kategori

In [15]:
BASE_URL = "https://www.cnnindonesia.com/"
headers = {"User-Agent": "Mozilla/5.0"}

def get_categories():
    response = requests.get(BASE_URL, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    categories = []
    for a in soup.select("a.navbar__item"):
        href = a.get("href")
        if href and href.startswith(BASE_URL):
            slug = href.replace(BASE_URL, "").strip("/")
            if slug and slug not in categories:
                categories.append(slug)
    return categories

def scrape_category(category, max_items=20):
    url = BASE_URL + category
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, "html.parser")

    data = []
    # cari link berita di halaman kategori
    links = set()
    for a in soup.select("article a, .list-berita a, .headline a, .container__list__desc a"):
        href = a.get("href")
        if href and href.startswith("http"):
            links.add(href)

    # ambil maksimal `max_items`
    for link in list(links)[:max_items]:
        try:
            resp_detail = requests.get(link, headers=headers)
            soup_detail = BeautifulSoup(resp_detail.text, "html.parser")

            # judul
            judul_tag = soup_detail.find("h1")
            judul = judul_tag.get_text(strip=True) if judul_tag else ""

            # kategori (pakai slug pertama di URL)
            kategori = None
            try:
                path = link.replace(BASE_URL, "")
                kategori = path.split("/")[0] if path else None
            except Exception:
                kategori = None

            # isi berita
            paragraf = [p.get_text(strip=True) for p in soup_detail.select("div.detail-text p")]
            isi = " ".join(paragraf)

            if isi:
                data.append({
                    "judul": judul,
                    "kategori": kategori,
                    "isi": isi,
                    "link": link
                })

            time.sleep(0.5)

        except Exception as e:
            print(f"Gagal ambil {link}: {e}")
    return data

if __name__ == "__main__":
    all_data = []
    categories = get_categories()
    print("Kategori ditemukan:", categories)

    for cat in categories:
        print(f"Scraping kategori {cat} ...")
        items = scrape_category(cat, max_items=20)
        all_data.extend(items)
        print(f"  → {len(items)} berita diambil dari {cat}")
        time.sleep(1)

    # simpan ke DataFrame
    df = pd.DataFrame(all_data)
    pd.set_option("display.max_colwidth", 100)
    print(df.head(10))  # tampilkan 10 berita pertama

    df.to_csv("berita_cnn.csv", index=False, encoding="utf-8-sig")
    print("Selesai, data disimpan ke berita_cnn.csv")

Kategori ditemukan: ['nasional', 'internasional', 'ekonomi', 'olahraga', 'teknologi', 'otomotif', 'hiburan', 'gaya-hidup']
Scraping kategori nasional ...
  → 18 berita diambil dari nasional
Scraping kategori internasional ...
  → 19 berita diambil dari internasional
Scraping kategori ekonomi ...
  → 20 berita diambil dari ekonomi
Scraping kategori olahraga ...
  → 20 berita diambil dari olahraga
Scraping kategori teknologi ...
  → 19 berita diambil dari teknologi
Scraping kategori otomotif ...
  → 19 berita diambil dari otomotif
Scraping kategori hiburan ...
  → 19 berita diambil dari hiburan
Scraping kategori gaya-hidup ...
  → 17 berita diambil dari gaya-hidup
                                                                  judul  \
0       Ada Putusan MK, TNI Masih Pertimbangkan Polisikan Ferry Irwandi   
1                                Sara Keponakan Prabowo Mundur dari DPR   
2                   Tanggul Beton Cilincing, DKI Sebut Perizinan di KKP   
3          Pernyataan Lengkap

In [16]:
df

,judul,kategori,isi,link
0,"Ada Putusan MK, TNI Masih Pertimbangkan Polisikan Ferry Irwandi",nasional,TNImasih mempertimbangkan langkah hukum yang akan diambil terhadap CEO Malaka ProjectFerry Irwan...,https://www.cnnindonesia.com/nasional/20250910145755-12-1272165/ada-putusan-mk-tni-masih-pertimb...
1,Sara Keponakan Prabowo Mundur dari DPR,nasional,Politikus Partai GerindraRahayu Saraswati Djojohadikusumoalias Sara memutuskan mundur sebagai an...,https://www.cnnindonesia.com/nasional/20250910182921-32-1272266/sara-keponakan-prabowo-mundur-da...
2,"Tanggul Beton Cilincing, DKI Sebut Perizinan di KKP",nasional,Staf Khusus Gubernur DKI Jakarta Bidang Komunikasi Sosial Chico Hakim mengatakan perizinan terka...,https://www.cnnindonesia.com/nasional/20250910171358-20-1272247/tanggul-beton-cilincing-dki-sebu...
3,Pernyataan Lengkap Rahayu Saraswati Gerindra Mundur dari DPR,nasional,"Politikus Partai Gerindrayang juga keponakan Presiden Prabowo Subianto,Rahayu Saraswati Djojohad...",https://www.cnnindonesia.com/nasional/20250910190234-32-1272282/pernyataan-lengkap-rahayu-sarasw...
4,Rahayu Saraswati Ungkap Alasan Mundur dari DPR,nasional,"Keponakan Presiden Prabowo Subianto,Rahayu Saraswati Djojohadikusumomengungkap alasan dirinya mu...",https://www.cnnindonesia.com/nasional/20250910185904-32-1272281/rahayu-saraswati-ungkap-alasan-m...
...,...,...,...,...
146,"DKI Evaluasi Cakupan Imunisasi Campak Hingga Tingkat RT, Ini Alasannya",gaya-hidup,DKI evaluasi cakupanimunisasicampak hingga tingkat RT sebagai langkah serius menekan penyebaranp...,https://www.cnnindonesia.com/gaya-hidup/20250910100657-255-1272004/dki-evaluasi-cakupan-imunisas...
147,Apa yang Terjadi pada Tubuh Saat Stres Berlebihan?,gaya-hidup,Situasi negara saat ini tak pelak bikinstres. Stres terus-menerus atau berlebihan tidak bisa dia...,https://www.cnnindonesia.com/gaya-hidup/20250903103057-255-1269571/apa-yang-terjadi-pada-tubuh-s...
148,"5 Sayuran Ini Tak Dianjurkan untuk Penderita Asam Urat, Siapa Saja?",gaya-hidup,"Banyak pakarkesehatanmenganjurkan konsumsisayurandemi kesehatan jangka panjang. Namun, ada beber...",https://www.cnnindonesia.com/gaya-hidup/20250829102742-255-1267731/5-sayuran-ini-tak-dianjurkan-...
149,"Marak Delay, Maskapai Ini Persilakan Penumpang 'Serbu' Email Menteri",gaya-hidup,MaskapaiRyanair mengimbau para penumpang yang mengalami penundaan penerbangan akibat masalah pen...,https://www.cnnindonesia.com/gaya-hidup/20250908144448-269-1271253/marak-delay-maskapai-ini-pers...
